# FAISS Vectore Store Setup

### Imports

In [ ]:
%pip install llama-index-vector-stores-faiss

In [ ]:
%pip install faiss-cpu

### Load credentials

In [ ]:
import os
from getpass import getpass

if "LLAMA_CLOUD_API_KEY" not in os.environ:
    os.environ["LLAMA_CLOUD_API_KEY"] = getpass("Enter your Llama Cloud API Key: ")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

### Setup faiss index

In [2]:
import faiss

# dimensions of text-ada-embedding-002/text-embedding-3-small
d = 1536
faiss_index = faiss.IndexFlatL2(d)

### Load documents, build the VectorStoreIndex

In [8]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

# load documents
documents = SimpleDirectoryReader("/home/daghbeji/rag-factory/sandbox/vector-stores/data/").load_data()
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

2026-02-21 12:19:30,340 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [9]:
# save index to disk
index.storage_context.persist()

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage_faiss")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage_faiss"
)
index = load_index_from_storage(storage_context=storage_context)

2026-02-21 12:19:33,577 - INFO - Loading llama_index.vector_stores.faiss.base from ./storage/default__vector_store.json.
2026-02-21 12:19:33,578 - INFO - Loading all indices.


### Query Index

In [7]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("How many file are in our batch?")
display(Markdown(f"<b>{response}</b>"))

response = query_engine.query("Summarize in 3-4 sentenecs.")
display(Markdown(f"<b>{response}</b>"))

2026-02-21 12:18:49,523 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-21 12:18:50,563 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Two files are in the batch.</b>

2026-02-21 12:18:50,822 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-21 12:18:51,784 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>A new framework called RAG-Anything is introduced to address the limitations of existing systems in handling multimodal knowledge representation and retrieval. It employs a dual-graph construction strategy to integrate visual elements, structured data, and textual knowledge seamlessly. By combining structural knowledge navigation with semantic similarity matching, RAG-Anything overcomes the shortcomings of traditional approaches and achieves superior performance, especially on long-context documents with evidence spanning multiple modalities.</b>

In [12]:
response = query_engine.query("How many times was the word RAG used? Give me the lines at which the word occurs.")
display(Markdown(f"<b>{response}</b>"))

2026-02-21 14:24:45,678 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-21 14:24:48,769 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>The word "RAG" was used 18 times in the provided context. Here are the lines at which the word occurs:
1. RAG-ANYTHING: ALL-IN-ONE RAG FRAMEWORK
2. Current RAG frameworks systematically exclude these vital knowledge sources across all three scenarios.
3. Therefore, multimodal RAG emerges as a critical advancement.
4. Addressing multimodal RAG presents three fundamental technical challenges that demand principled solutions.
5. This makes it significantly more complex than traditional text-only approaches.
6. The naive solution of converting all multimodal content to textual descriptions introduces severe information loss.
7. These inherent limitations necessitate the design of effective technical components.
8. Technical Challenges. • First, the unified multimodal representation challenge requires seamlessly integrating diverse information types.
9. Second, the structure-aware decomposition challenge demands intelligent parsing of complex layouts.
10. Third, the cross-modal retrieval challenge necessitates sophisticated mechanisms that can navigate between different modalities.
11. Our Contributions. To address these challenges, we introduce RAG-Anything, a unified framework that fundamentally reimagines multimodal knowledge representation and retrieval.
12. Rather than forcing diverse modalities into text-centric pipelines, RAG-Anything constructs complementary knowledge graphs that preserve both multimodal contextual relationships and detailed textual knowledge.
13. The system maintains semantic integrity across modalities while ensuring efficient cross-modal reasoning capabilities throughout the process.
14. Our cross-modal hybrid retrieval mechanism strategically combines structural knowledge navigation with semantic similarity matching.
15. RAG-Anything leverages explicit graph relationships to capture multi-hop reasoning patterns.
16. The framework introduces modality-aware query processing and cross-modal alignment systems.
17. This unified approach eliminates the architectural fragmentation that plagues current multimodal RAG systems.
18. Our evaluation demonstrates that RAG-Anything achieves superior performance across diverse domains.</b>